In [37]:
# 1. 표준 라이브러리
import sys
from pathlib import Path
import platform
import time
import json
import os
import joblib
from datetime import datetime
from typing import Optional
from dataclasses import dataclass


# 2. 서드파티 라이브러리 

# 2-1. 시각화
import plotly.express as px

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns

# 2-2. 
import shap
from imblearn.pipeline import Pipeline as ImbPipeline
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, make_scorer, recall_score, 
    precision_score, f1_score, fbeta_score, average_precision_score, balanced_accuracy_score, precision_recall_fscore_support
)
from scipy.stats import uniform, randint
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.base import BaseEstimator

sys.path.append(str(Path.cwd().parent))
from utils import DATA_DIR, MODEL_DIR

In [38]:
# 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False

mpl.rcParams['axes.unicode_minus'] = False
# 폰트 개인 경로에 맞춰서 변경
# FONT_DIR = Path("/path/to/fonts")
# font_path = FONT_DIR / 'FREESENTATION-6SEMIBOLD.ttf'
# prop = fm.FontProperties(fname=font_path)

In [39]:
pandas_kwargs = {
    'parse_dates': ['검정일자'],
    'date_format': '%Y-%m-%d'
}

milk: pd.DataFrame = pd.read_csv(DATA_DIR /'interim' / 'milk.csv', **pandas_kwargs)
milk.head()

,농장아이디,개체번호,검정일자,누적착유일(연계),무지고형분,MUN,305일유량,305일무지고형분,전산차비유지속성,전산차건유전유량,...,계절,농장구분,정액코드분류,가격,가격구분,분만간격,분만월령,공태일수_log,계절_sin,비유단계_sin
0,20249,20120709020022,2020-01-11,154,0,1,9714.0,805.0,50.5,16.1,...,4,1,NaN,1056.34,1,NaN,85,NaN,-2.449294e-16,1.000000e+00
1,20249,20120504020095,2020-01-11,52,0,1,NaN,NaN,68.8,22.6,...,4,1,NaN,1056.34,1,NaN,90,NaN,-2.449294e-16,1.000000e+00
2,20249,20111008020210,2020-01-11,115,0,1,11008.0,971.0,54.1,24.5,...,4,1,NaN,1060.34,1,NaN,95,NaN,-2.449294e-16,1.224647e-16
3,20249,20121014020049,2020-01-11,290,1,1,11318.0,1011.0,58.6,37.8,...,4,1,7HO,1062.31,1,NaN,77,5.4161,-2.449294e-16,-1.000000e+00
4,20249,20130812020216,2020-01-11,100,0,1,11324.0,957.0,68.1,20.5,...,4,1,NaN,1056.34,1,NaN,75,NaN,-2.449294e-16,1.224647e-16


In [40]:
df = milk

In [41]:
features = [
    # '누적착유일(연계)',
    '전산차비유지속성',
    '산차', 
    '농후사료급여량(연계)',
    # '공태일수',
    '공태일수_log',
    # '305일유량',
    '농장구분',
    # '분만간격',
    # '분만월령',
    # '비유단계',
    '비유단계_sin',
    # '계절',
    '계절_sin'
    # '정액코드분류'
]

target = '가격구분'

In [42]:
df[target].value_counts(normalize=True).mul(100).round()

가격구분
1    56.0
0    33.0
2    11.0
Name: proportion, dtype: float64

## Train Test Split

In [43]:
split_date = '2021-08-01'
train = df[df['검정일자'] < split_date]
test = df[df['검정일자'] >= split_date]

X_train, X_test = train[features], test[features]
y_train, y_test = train[target], test[target]

In [44]:
train.shape, test.shape

((31743, 25), (8521, 25))

In [45]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pkg_resources")

# ============================================
# 1. 클래스 비율 확인 (Multiclass)
# ============================================
print("="*60)
print("클래스 분포 분석 (Multiclass)")
print("="*60)
print(y_train.value_counts().sort_index())
print(f"\n비율:")
print(y_train.value_counts(normalize=True).sort_index())

# 🔥 Multiclass: 3개 클래스 (비율 대략 1:5:3)
# 0: C등급 (소수, ~11%)
# 1: B등급 (다수, ~56%)
# 2: A등급 (중간, ~33%)
class_counts = y_train.value_counts().sort_index()
n_classes = len(class_counts)

print(f"\n클래스 구성 ({n_classes}개):")
class_labels = {
    0: 'A등급 (중간)',    # 🔥 수정
    1: 'B등급 (다수)',   # 🔥 수정
    2: 'C등급 (소수)'   # 🔥 수정
}

for cls in sorted(class_counts.index):
    count = class_counts[cls]
    pct = count / len(y_train) * 100
    label = class_labels.get(cls, f'클래스{cls}')
    print(f"   {cls} ({label}): {count}개 ({pct:.1f}%)")

# 🔥 class_weight 계산 (불균형 대응)
from sklearn.utils.class_weight import compute_class_weight

class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = {i: w for i, w in enumerate(class_weights_array)}

print(f"\n계산된 class_weight (불균형 보정):")
for cls, weight in class_weight_dict.items():
    label = class_labels[cls]
    print(f"   클래스 {cls} ({label}): {weight:.2f}")

# 결측치 확인
print("\n결측치 현황:")
missing_info = pd.DataFrame({
    'Feature': X_train.columns,
    '결측_개수': X_train.isnull().sum().values,
    '결측_비율(%)': (X_train.isnull().mean() * 100).values
}).sort_values('결측_비율(%)', ascending=False)
print(missing_info.head(10))
print(f"\n전체 결측 비율: {X_train.isnull().mean().mean()*100:.1f}%")
print(f"Feature 개수: {X_train.shape[1]}개")

# ============================================
# 2. Pipeline 구성 (Multiclass)
# ============================================
from imblearn.pipeline import Pipeline as ImbPipeline
from lightgbm import LGBMClassifier

pipeline = ImbPipeline([
    ('model', LGBMClassifier(
        objective='multiclass',  # 🔥 변경
        num_class=n_classes,     # 🔥 추가
        device='gpu',
        random_state=42,
        verbosity=-1,
        n_jobs=5,
        class_weight='balanced',  # 🔥 변경 (또는 class_weight_dict)
        force_col_wise=True,
        metric='None',
    ))
])

print("\n✅ Pipeline 구성:")
print("   1. LGBMClassifier (GPU, Multiclass)")
print(f"      - num_class={n_classes}")
print(f"      - class_weight='balanced'")

# ============================================
# 3. 파라미터 분포
# ============================================
from scipy.stats import uniform, randint

param_distributions = {
    'model__n_estimators': randint(600, 1000),
    'model__learning_rate': uniform(0.005, 0.045),
    'model__max_depth': randint(3, 8),
    'model__num_leaves': randint(15, 80),
    'model__min_child_samples': randint(5, 50),
    'model__subsample': uniform(0.6, 0.4),
    'model__colsample_bytree': uniform(0.6, 0.4),
    'model__min_child_weight': [1e-5, 1e-4, 1e-3, 1e-2],
    'model__reg_alpha': uniform(0, 0.5),
    'model__reg_lambda': uniform(0, 0.5),
}

# ============================================
# 4. Scoring 설정 (Multiclass)
# ============================================
from sklearn.metrics import (
    make_scorer, recall_score, precision_score, f1_score, 
    fbeta_score, roc_auc_score, accuracy_score
)

# 🔥 Multiclass용 scorer
scoring = {
    'accuracy': 'accuracy',
    'recall_macro': make_scorer(recall_score, average='macro'),          # 🔥 macro 평균
    'recall_weighted': make_scorer(recall_score, average='weighted'),    # 🔥 weighted 평균
    'precision_macro': make_scorer(precision_score, average='macro', zero_division=0),
    'precision_weighted': make_scorer(precision_score, average='weighted', zero_division=0),
    'f1_macro': make_scorer(f1_score, average='macro', zero_division=0),
    'f1_weighted': make_scorer(f1_score, average='weighted', zero_division=0),
    'roc_auc_ovr': make_scorer(roc_auc_score, multi_class='ovr', response_method='predict_proba'),  # 🔥 One-vs-Rest
}

main_score = 'f1_macro'  # 🔥 변경 (또는 'f1_weighted', 'accuracy')

print("\n✅ Scoring 전략 (Multiclass):")
print(f"   메인 목표: {main_score.upper()}")
print(f"   평가: macro-average (모든 클래스 동등 가중)")

# ============================================
# 5. Stratified K-Fold
# ============================================
from sklearn.model_selection import StratifiedKFold

n_folds = 5
n_iter = 300
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

print(f"\n교차검증 Folds: {n_folds}")
print(f"랜덤 샘플링 조합 수: {n_iter}개")
print(f"총 Fits: {n_iter} × {n_folds} = {n_iter * n_folds}")

# ============================================
# 6. RandomizedSearchCV 실행
# ============================================
from sklearn.model_selection import RandomizedSearchCV
import time
import numpy as np

print("\n" + "="*60)
print("🚀 RandomizedSearchCV 시작 (Multiclass)!")
print("="*60)

lgbm_random = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=cv,
    scoring=scoring,
    refit=main_score,
    verbose=3,
    random_state=42,
    n_jobs=5,
    error_score=np.nan
)

start_actual = time.time()
lgbm_random.fit(X_train, y_train)
actual_time = time.time() - start_actual

# ============================================
# 7. 결과 출력
# ============================================
import pandas as pd

print("\n" + "="*60)
print("✅ 완료!")
print("="*60)
print(f"⏱️  소요 시간: {actual_time/60:.1f}분 ({actual_time:.0f}초)")

print(f"\n🏆 최고 {main_score.upper()} (CV): {lgbm_random.best_score_:.4f}")

print("\n📋 최적 파라미터:")
for key, value in sorted(lgbm_random.best_params_.items()):
    if isinstance(value, float):
        print(f"   {key}: {value:.4f}")
    else:
        print(f"   {key}: {value}")

# ============================================
# 8. 전체 결과 비교
# ============================================
results_df = pd.DataFrame(lgbm_random.cv_results_)

comparison_cols = [
    f'rank_test_{main_score}',
    'mean_test_accuracy',
    'mean_test_f1_macro',
    'mean_test_f1_weighted',
    'mean_test_recall_macro',
    'mean_test_precision_macro',
    'mean_test_roc_auc_ovr',
    'params'
]

print("\n" + "="*60)
print(f"📊 상위 10개 파라미터 조합 ({main_score.upper()} 기준)")
print("="*60)
top_10 = results_df[comparison_cols].sort_values(f'rank_test_{main_score}').head(10)

for idx, row in top_10.iterrows():
    rank = int(row[f'rank_test_{main_score}'])
    main_metric = row[f'mean_test_{main_score}']
    acc = row['mean_test_accuracy']
    f1_macro = row['mean_test_f1_macro']
    f1_weighted = row['mean_test_f1_weighted']
    
    print(f"\n#{rank} - {main_score.upper()}: {main_metric:.4f}")
    print(f"   Accuracy: {acc:.4f} | F1-Macro: {f1_macro:.4f} | F1-Weighted: {f1_weighted:.4f}")
    
    params = row['params']
    print(f"   Model: lr={params.get('model__learning_rate', 0):.4f}, "
          f"depth={params.get('model__max_depth', 0)}, "
          f"leaves={params.get('model__num_leaves', 0)}")

# ============================================
# 9. 테스트 세트 평가 (Multiclass)
# ============================================
from sklearn.metrics import classification_report, confusion_matrix

print("\n" + "="*60)
print("🧪 테스트 세트 평가 (Multiclass)")
print("="*60)

best_model = lgbm_random.best_estimator_
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)  # 🔥 shape: (n_samples, n_classes)

# 분류 리포트
target_names = [class_labels.get(i, f'클래스{i}') for i in range(n_classes)]
print(classification_report(y_test, y_pred, target_names=target_names))

# 혼동 행렬 (3x3)
print("\n혼동 행렬:")
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm,
    index=[f'실제_{label}' for label in target_names],
    columns=[f'예측_{label}' for label in target_names]
)
print(cm_df)

# ============================================
# 10. 클래스별 성능 분석
# ============================================
print("\n" + "="*60)
print("📊 클래스별 상세 성능")
print("="*60)

from sklearn.metrics import precision_recall_fscore_support

precisions, recalls, f1s, supports = precision_recall_fscore_support(
    y_test, y_pred, average=None
)

for i in range(n_classes):
    label = target_names[i]
    print(f"\n클래스 {i} ({label}):")
    print(f"   Precision: {precisions[i]:.4f}")
    print(f"   Recall:    {recalls[i]:.4f}")
    print(f"   F1-score:  {f1s[i]:.4f}")
    print(f"   Support:   {supports[i]} 샘플")

# 전체 메트릭
print("\n" + "="*60)
print("📈 전체 성능 메트릭")
print("="*60)

test_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'F1-Macro': f1_score(y_test, y_pred, average='macro'),
    'F1-Weighted': f1_score(y_test, y_pred, average='weighted'),
    'Recall-Macro': recall_score(y_test, y_pred, average='macro'),
    'Precision-Macro': precision_score(y_test, y_pred, average='macro', zero_division=0),
    'ROC-AUC-OVR': roc_auc_score(y_test, y_pred_proba, multi_class='ovr')
}

for metric_name, value in test_metrics.items():
    print(f"{metric_name:<20}: {value:.4f}")

# ============================================
# 11. 혼동 행렬 시각화 (선택 사항)
# ============================================
print("\n💡 혼동 행렬 해석:")
total_correct = np.trace(cm)
total_samples = cm.sum()
print(f"   정확히 분류된 샘플: {total_correct}/{total_samples} ({total_correct/total_samples*100:.1f}%)")

for i in range(n_classes):
    total_actual = cm[i, :].sum()
    correct = cm[i, i]
    misclassified = total_actual - correct
    print(f"   클래스 {i} ({target_names[i]}): {correct}/{total_actual} 정확 ({misclassified}개 오분류)")

# ============================================
# 12. 변수 중요도 확인
# ============================================
print("\n" + "="*60)
print("📊 변수 중요도 Top 10")
print("="*60)

final_model = best_model.named_steps['model']
feature_names = list(X_train.columns)

importance_df = pd.DataFrame({
    'feature': feature_names[:len(final_model.feature_importances_)],
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print(importance_df.head(10).to_string(index=False))

# ============================================
# 13. 교차검증 vs 테스트 성능 비교
# ============================================
print("\n" + "="*60)
print("📈 성능 비교: 교차검증 vs 테스트")
print("="*60)

print(f"{'지표':<20} {'교차검증 (CV)':<20} {'테스트':<15} {'차이':<10}")
print("-" * 70)

cv_metrics = {
    'Accuracy': 'accuracy',
    'F1-Macro': 'f1_macro',
    'F1-Weighted': 'f1_weighted',
    'Recall-Macro': 'recall_macro',
    'Precision-Macro': 'precision_macro',
    'ROC-AUC-OVR': 'roc_auc_ovr'
}

for display_name, cv_key in cv_metrics.items():
    cv_score = results_df.loc[
        results_df[f'rank_test_{main_score}'] == 1, 
        f'mean_test_{cv_key}'
    ].values[0]
    test_score = test_metrics[display_name]
    diff = test_score - cv_score
    
    diff_str = f"{diff:+.4f}"
    if abs(diff) > 0.05:
        diff_str += " ⚠️"
    
    print(f"{display_name:<20} {cv_score:<20.4f} {test_score:<15.4f} {diff_str}")

# ============================================
# 14. 클래스별 ROC-AUC (선택 사항)
# ============================================
print("\n" + "="*60)
print("📊 클래스별 ROC-AUC (One-vs-Rest)")
print("="*60)

from sklearn.metrics import roc_auc_score

for i in range(n_classes):
    # 이진화: 클래스 i vs 나머지
    y_test_binary = (y_test == i).astype(int)
    y_pred_proba_class = y_pred_proba[:, i]
    
    auc = roc_auc_score(y_test_binary, y_pred_proba_class)
    print(f"클래스 {i} ({target_names[i]}): {auc:.4f}")

# ============================================
# 15. 최종 요약
# ============================================
print("\n" + "="*60)
print(f"📝 최종 요약 ({main_score.upper()} 최적화)")
print("="*60)
print(f"✅ 최적 모델 {main_score.upper()} (CV): {lgbm_random.best_score_:.4f}")
print(f"✅ 테스트 Accuracy: {test_metrics['Accuracy']*100:.1f}%")
print(f"✅ 테스트 F1-Macro: {test_metrics['F1-Macro']:.4f}")
print(f"✅ 테스트 F1-Weighted: {test_metrics['F1-Weighted']:.4f}")
print(f"⏱️  총 소요 시간: {actual_time/60:.1f}분")

print("\n💡 핵심 포인트:")
for i in range(n_classes):
    print(f"   - 클래스 {i} ({target_names[i]}): Precision={precisions[i]:.3f}, Recall={recalls[i]:.3f}, F1={f1s[i]:.3f}")

# ============================================
# 16. 모델 + 파라미터 저장
# ============================================
import joblib
from datetime import datetime
from pathlib import Path
import json

print("\n" + "="*60)
print("💾 모델 및 파라미터 저장")
print("="*60)

save_model_input = input("모델을 저장하시겠습니까? (y/n): ")
if save_model_input.lower() == 'y':
    try:
        MODEL_DIR = Path(MODEL_DIR)
    except:
        MODEL_DIR = Path('.')
        print(f"⚠️  MODEL_DIR 미정의, 현재 디렉토리 사용: {MODEL_DIR.absolute()}")
    
    MODEL_DIR.mkdir(exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    model_filename = MODEL_DIR / f'best_lgbm_multiclass_{main_score}_{timestamp}.pkl'
    params_filename = MODEL_DIR / f'best_params_multiclass_{timestamp}.json'
    metrics_filename = MODEL_DIR / f'test_metrics_multiclass_{timestamp}.json'
    results_filename = MODEL_DIR / f'cv_results_multiclass_{timestamp}.csv'
    
    # 모델 저장
    joblib.dump(best_model, model_filename)
    print(f"✅ 모델 저장: {model_filename}")
    
    # 파라미터 저장
    best_params_serializable = {}
    for key, value in lgbm_random.best_params_.items():
        if isinstance(value, (np.integer, np.floating)):
            best_params_serializable[key] = float(value) if isinstance(value, np.floating) else int(value)
        else:
            best_params_serializable[key] = value
    
    params_info = {
        'best_params': best_params_serializable,
        f'best_{main_score}_score': float(lgbm_random.best_score_),
        'optimization_target': main_score,
        'cv_folds': n_folds,
        'n_iter': n_iter,
        'timestamp': timestamp,
        'n_classes': n_classes,
        'class_labels': class_labels,
        'class_distribution': {int(k): int(v) for k, v in class_counts.items()},
        'class_weights': {int(k): float(v) for k, v in class_weight_dict.items()}
    }
    
    with open(params_filename, 'w', encoding='utf-8') as f:
        json.dump(params_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 파라미터 저장: {params_filename}")
    
    # 테스트 성능 저장
    test_metrics_serializable = {
        key: float(value) for key, value in test_metrics.items()
    }
    
    metrics_info = {
        'test_metrics': test_metrics_serializable,
        'confusion_matrix': cm.tolist(),
        'class_performance': {
            int(i): {
                'precision': float(precisions[i]),
                'recall': float(recalls[i]),
                'f1': float(f1s[i]),
                'support': int(supports[i])
            }
            for i in range(n_classes)
        },
        'timestamp': timestamp
    }
    
    with open(metrics_filename, 'w', encoding='utf-8') as f:
        json.dump(metrics_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 테스트 성능 저장: {metrics_filename}")
    
    # CV 결과 저장
    results_df.to_csv(results_filename, index=False, encoding='utf-8')
    print(f"✅ CV 결과 저장: {results_filename}")
    
    print("\n📦 저장된 파일 요약:")
    print(f"1. 모델: {model_filename.name}")
    print(f"2. 파라미터: {params_filename.name}")
    print(f"3. 성능: {metrics_filename.name}")
    print(f"4. CV결과: {results_filename.name}")
else:
    print("모델 저장 생략")

print("\n" + "="*60)
print("🎉 모든 작업 완료 (Multiclass)!")
print("="*60)

클래스 분포 분석 (Multiclass)
가격구분
0    10587
1    17819
2     3337
Name: count, dtype: int64

비율:
가격구분
0    0.333522
1    0.561352
2    0.105126
Name: proportion, dtype: float64

클래스 구성 (3개):
   0 (A등급 (중간)): 10587개 (33.4%)
   1 (B등급 (다수)): 17819개 (56.1%)
   2 (C등급 (소수)): 3337개 (10.5%)

계산된 class_weight (불균형 보정):
   클래스 0 (A등급 (중간)): 1.00
   클래스 1 (B등급 (다수)): 0.59
   클래스 2 (C등급 (소수)): 3.17

결측치 현황:
       Feature  결측_개수   결측_비율(%)
0     전산차비유지속성  18010  56.736918
3     공태일수_log  13326  41.980909
1           산차      0   0.000000
2  농후사료급여량(연계)      0   0.000000
4         농장구분      0   0.000000
5     비유단계_sin      0   0.000000
6       계절_sin      0   0.000000

전체 결측 비율: 14.1%
Feature 개수: 7개

✅ Pipeline 구성:
   1. LGBMClassifier (GPU, Multiclass)
      - num_class=3
      - class_weight='balanced'

✅ Scoring 전략 (Multiclass):
   메인 목표: F1_MACRO
   평가: macro-average (모든 클래스 동등 가중)

교차검증 Folds: 5
랜덤 샘플링 조합 수: 300개
총 Fits: 300 × 5 = 1500

🚀 RandomizedSearchCV 시작 (Multiclass)!
Fitting 5 folds for eac

/home/dataiku/eric/anaconda3/envs/smartfarm/lib/python3.10/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/dataiku/eric/anaconda3/envs/smartfarm/lib/python3.10/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/dataiku/eric/anaconda3/envs/smartfarm/lib/python3.10/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2

[CV 3/5] END model__colsample_bytree=0.749816047538945, model__learning_rate=0.047782143788446224, model__max_depth=5, model__min_child_samples=12, model__min_child_weight=1e-05, model__n_estimators=620, model__num_leaves=38, model__reg_alpha=0.32544423647442644, model__reg_lambda=0.028205789513550128, model__subsample=0.8887995089067299; accuracy: (test=0.547) f1_macro: (test=0.504) f1_weighted: (test=0.560) precision_macro: (test=0.504) precision_weighted: (test=0.612) recall_macro: (test=0.550) recall_weighted: (test=0.547) roc_auc_ovr: (test=0.725) total time=  22.2s
[CV 2/5] END model__colsample_bytree=0.749816047538945, model__learning_rate=0.047782143788446224, model__max_depth=5, model__min_child_samples=12, model__min_child_weight=1e-05, model__n_estimators=620, model__num_leaves=38, model__reg_alpha=0.32544423647442644, model__reg_lambda=0.028205789513550128, model__subsample=0.8887995089067299; accuracy: (test=0.542) f1_macro: (test=0.498) f1_weighted: (test=0.555) precision

In [46]:
models = [
    'best_lgbm_pr_auc_optimized_20251108_221353'
    'best_lgbm_pr_auc_optimized_20251106_215224',
    'best_lgbm_pr_auc_optimized_20251108_223541',
    'best_lgbm_pr_auc_optimized_20251108_225820',
    'best_lgbm_precision_optimized_20251108_232046',
    'best_lgbm_f1_optimized_20251108_235801',
    'best_lgbm_pr_auc_optimized_20251109_001812',
    'best_lgbm_pr_auc_optimized_20251109_004121',
    'best_lgbm_precision_optimized_20251109_012906',
    'best_lgbm_precision_optimized_20251109_021948', 
    'best_lgbm_precision_optimized_20251109_023236',
    'best_lgbm_precision_optimized_20251109_041233',
    'best_lgbm_precision_optimized_20251109_090643',
    'best_lgbm_multiclass_f1_macro_20251109_163631'
]

In [47]:
import joblib
loaded_model = joblib.load(MODEL_DIR / 'best_lgbm_multiclass_f1_macro_20251109_163631.pkl')

In [ ]:
from sklearn.metrics import precision_recall_curve, auc
from sklearn.preprocessing import label_binarize

# 클래스 이름과 이진화
classes = df[target].unique()
y_test_bin = label_binarize(y_test, classes=classes)

# 모델 확률 예측
y_prob = loaded_model.predict_proba(X_test)  # shape = (n_samples, n_classes)

# 각 클래스별 최적 threshold 찾기
best_thresholds = {}
for i, cls in enumerate(classes):
    best_f1 = 0
    best_t = 0
    thresholds_to_try = np.linspace(0.1, 0.9, 81)  # 0.1~0.9까지 0.01 간격
    for t in thresholds_to_try:
        y_pred_cls = (y_prob[:, i] >= t).astype(int)
        f1 = f1_score(y_test_bin[:, i], y_pred_cls)
        if f1 > best_f1:
            best_f1 = f1
            best_t = t
    best_thresholds[cls] = best_t
    print(f"Class {cls}: Best Threshold = {best_t:.2f}, Best F1 = {best_f1:.4f}")

# 최적 threshold 적용 후 멀티클래스 예측
y_pred_thresholded = []
for p in y_prob:
    # threshold 미만 확률은 0으로
    adjusted_p = np.array([prob if prob >= best_thresholds[cls] else 0
                           for cls, prob in zip(classes, p)])
    # 최대값 선택
    y_pred_thresholded.append(classes[np.argmax(adjusted_p)])
y_pred = np.array(y_pred_thresholded)

# 평가
from sklearn.metrics import classification_report, balanced_accuracy_score, confusion_matrix

print("\nClassification Report (Threshold Optimized):")
print(classification_report(y_test, y_pred))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))
roc_auc_ovo = roc_auc_score(y_test_bin, y_prob, multi_class='ovo', average='macro') # <- 조정해도 값이 바뀔 수 없는 건가? 순서 기준으로 점수를 매겨서 임계값과
print(f"ROC-AUC (OvO, macro, Test Set): {roc_auc_ovo:.3f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# PR-AUC 계산
pr_auc_dict = {}
for i, cls in enumerate(classes):
    precision, recall, _ = precision_recall_curve(y_test_bin[:, i], y_prob[:, i])
    pr_auc = auc(recall, precision)
    pr_auc_dict[cls] = pr_auc
    print(f"Class {cls} - PR-AUC: {pr_auc:.4f}")

pr_auc_macro = np.mean(list(pr_auc_dict.values()))
print(f"Macro-average PR-AUC: {pr_auc_macro:.4f}")

y_pred = np.array(y_pred)

Class 1: Best Threshold = 0.10, Best F1 = 0.6683
Class 0: Best Threshold = 0.10, Best F1 = 0.4920
Class 2: Best Threshold = 0.40, Best F1 = 0.2946
CLS: 2

Classification Report (Threshold Optimized):
              precision    recall  f1-score   support

           0       0.25      0.33      0.28      2833
           1       0.45      0.32      0.37      4717
           2       0.25      0.35      0.29       971

    accuracy                           0.33      8521
   macro avg       0.32      0.33      0.32      8521
weighted avg       0.36      0.33      0.33      8521

Balanced Accuracy: 0.3334660752913649
ROC-AUC (OvO, macro, Test Set): 0.476
Confusion Matrix:
[[ 947 1563  323]
 [2534 1505  678]
 [ 355  279  337]]
Class 1 - PR-AUC: 0.4708
Class 0 - PR-AUC: 0.2618
Class 2 - PR-AUC: 0.2616
Macro-average PR-AUC: 0.3314


In [ ]:
thr_list = np.round(np.arange(0.1, 1.0, 0.01), 2)
rows = []
for thr in thr_list:
    y_hat = (y_prob >= thr).astype(int)
    rows.append([
        thr,
        precision_score(y_test, y_hat, zero_division=0),
        recall_score(y_test, y_hat, zero_division=0),
        f1_score(y_test, y_hat, zero_division=0)
    ])

thr_table = pd.DataFrame(rows, columns=['threshold', 'precision', 'recall', 'f1'])

fig = go.Figure()
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["precision"],
                         mode="lines", name="Precision",
                         line=dict(dash="dash", color="cyan")))
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["recall"],
                         mode="lines", name="Recall",
                         line=dict(color="magenta")))
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["f1"],
                         mode="lines", name="F1",
                         line=dict(color="yellow")))

fig.update_layout(title="임곗값 변화에 따른 Precision/Recall/F1",
                  xaxis_title="Threshold",
                  yaxis_title="Score",
                  yaxis=dict(range=[0,1]),
                  template="plotly_dark")
fig.show()

In [ ]:
import joblib
loaded_model = joblib.load(MODEL_DIR / 'best_lgbm_precision_optimized_20251109_041233.pkl')

params = loaded_model.named_steps['model'].get_params()

params['n_estimators'] = 1000000
params['learning_rate'] = 0.005
# params['num_leaves'] = 10
# params['max_depth'] = 5
# params['verbosity'] = 1
# params['metric'] = 'precision'

X_train2, X_valid, y_train2, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

display(params)

# pipeline = ImbPipeline([
#     ('model', LGBMClassifier(**params))
# ])

lgbm_model = LGBMClassifier(
    **params
)


def custom_precision(y_true, y_prob):
    y_pred = (y_prob >= 0.5).astype(int)
    return 'precision', precision_score(y_true, y_pred), True

lgbm_model.fit(
    X_train2, y_train2,
    eval_set=[(X_valid, y_valid)],  # train, valid 둘 다
    eval_names=['valid'],  # 이름 지정
    eval_metric=[custom_precision, 'average_precision'],  # PR-AUC (불균형 데이터에 적합)
    callbacks=[
        early_stopping(stopping_rounds=5000, verbose=True),
        log_evaluation(period=100)  # 100 iteration마다 출력
    ]
)


In [ ]:


# 테스트셋 성능 평가
y_pred = lgbm_model.predict(X_test)
y_prob = lgbm_model.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= 0.512).astype(int)

print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_pred, digits=3))

# Balanced Accuracy
bal_acc = balanced_accuracy_score(y_test, y_pred)
print(f"\nBalanced Accuracy (Test Set): {bal_acc:.3f}")

# ROC-AUC
roc_auc = roc_auc_score(y_test, y_prob)
pr_auc = average_precision_score(y_test, y_prob)
print(f"ROC-AUC (Test Set): {roc_auc:.3f}")
print(f"PR-AUC (Test Set): {pr_auc:.4f}")

In [ ]:
thr_list = np.round(np.arange(0.1, 1.0, 0.01), 2)
rows = []
for thr in thr_list:
    y_hat = (y_prob >= thr).astype(int)
    rows.append([
        thr,
        precision_score(y_test, y_hat, zero_division=0),
        recall_score(y_test, y_hat, zero_division=0),
        f1_score(y_test, y_hat, zero_division=0)
    ])

thr_table = pd.DataFrame(rows, columns=['threshold', 'precision', 'recall', 'f1'])

fig = go.Figure()
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["precision"],
                         mode="lines", name="Precision",
                         line=dict(dash="dash", color="cyan")))
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["recall"],
                         mode="lines", name="Recall",
                         line=dict(color="magenta")))
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["f1"],
                         mode="lines", name="F1",
                         line=dict(color="yellow")))

fig.update_layout(title="임곗값 변화에 따른 Precision/Recall/F1",
                  xaxis_title="Threshold",
                  yaxis_title="Score",
                  yaxis=dict(range=[0,1]),
                  template="plotly_dark")
fig.show()

In [ ]:
model = loaded_model.named_steps['model']

In [ ]:
import shap

# SHAP 적용
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# Plot
shap.summary_plot(shap_values, X_test)


In [ ]:
# mean absolute shap value per feature
shap_importance = np.abs(shap_values).mean(axis=0)
print(shap_importance)
shap_df = pd.DataFrame({
    'feature': X_test.columns,
    'shap_importance': shap_importance
}).sort_values(by='shap_importance', ascending=False)

print(shap_df)

## 집중학습

In [ ]:
# ============================================
# 2. Pipeline 구성
# ============================================

pipeline = ImbPipeline([
    ('model', LGBMClassifier(
        objective='binary',
        device='gpu',
        random_state=42,
        verbosity=-1,
        n_jobs=5,
        boost_from_average=False,
        # scale_pos_weight=scale_pos_weight,  # ✅ 자동으로 pos_label=1에 적용
        # bagging_freq=1,
        # pos_bagging_fraction=1,
        # neg_bagging_fraction=0.25,
        force_col_wise=True,
        metric='None',  # 🔥 추가: 기본 메트릭 끄기
    ))
])

print("\n✅ Pipeline 구성:")
print("   1. LGBMClassifier (GPU)")
print(f"      - scale_pos_weight={scale_pos_weight:.2f} (클래스 1=가격미달 가중치)")

# ============================================
# 3. 파라미터 분포
# ============================================
print(f"\n소수 클래스(1=가격미달) 샘플 수: {minority_count}개")

best_params = {
    "model__colsample_bytree": 0.6182968135250051,
    "model__learning_rate": 0.04421915627685693,
    "model__max_depth": 7,
    "model__min_child_samples": 16,
    "model__min_child_weight": 0.01,
    "model__n_estimators": 850,
    "model__num_leaves": 66,
    "model__reg_alpha": 0.3791315979645113,
    "model__reg_lambda": 0.012293458229400755,
    "model__scale_pos_weight": 5.199111963760975,
    "model__subsample": 0.7294440876598165
}

param_distributions = {
    # 🔥 LightGBM 파라미터 - 소수 클래스 민감도 향상
    'model__n_estimators': randint(800, 2000),
    'model__learning_rate': uniform(0.04, 0.05),
    'model__max_depth': randint(3, 6),
    'model__num_leaves': randint(15, 50),
    'model__min_child_samples': randint(20, 50),
    'model__subsample': uniform(0.7, 0.3),
    'model__colsample_bytree': uniform(0.6, 0.2),
    'model__min_child_weight': [0.01],
    'model__reg_alpha': uniform(0.3, 0.5),
    'model__reg_lambda': uniform(0, 0.1),
    
    # 불균형 처리
    'model__scale_pos_weight': uniform(5.0, 6.0),
    
    # 범주형 처리
    # 'model__cat_l2': uniform(5.0, 15.0), 
}


# ============================================
# 4. Scoring 설정
# ============================================

scoring = {
    'recall': make_scorer(recall_score),
    'f2': make_scorer(fbeta_score, beta=2),              
    'pr_auc': make_scorer(average_precision_score, response_method='predict_proba'),
    'precision': make_scorer(precision_score, zero_division=0),
    'f1': make_scorer(f1_score, zero_division=0, average='macro'),
    'roc_auc': 'roc_auc'                                           
}

main_score = 'precision'

print("\n✅ Scoring 전략:")
print(f"   메인 목표: {main_score.upper()}")
print(f"   타깃: 클래스 1 (가격미달) 검출 최적화")
print(f"   pos_label: 1 (기본값, 생략 가능)")

# ============================================
# 5. Stratified K-Fold
# ============================================
n_folds = 5
n_iter = 300
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)


print(f"\n교차검증 Folds: {n_folds}")
print(f"랜덤 샘플링 조합 수: {n_iter}개")
print(f"총 Fits: {n_iter} × {n_folds} = {n_iter * n_folds}")

# ============================================
# 6. RandomizedSearchCV 실행
# ============================================

print("\n" + "="*60)
print("🚀 RandomizedSearchCV 시작!")
print("="*60)

lgbm_random = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=cv,
    scoring=scoring,
    refit=main_score,
    verbose=3,
    random_state=42,
    n_jobs=5,
    error_score=np.nan 
)

start_actual = time.time()
lgbm_random.fit(X_train, y_train)
actual_time = time.time() - start_actual

# ============================================
# 7. 결과 출력
# ============================================
print("\n" + "="*60)
print("✅ 완료!")
print("="*60)
print(f"⏱️  소요 시간: {actual_time/60:.1f}분 ({actual_time:.0f}초)")

print(f"\n🏆 최고 {main_score.upper()} (CV): {lgbm_random.best_score_:.4f}")

print("\n📋 최적 파라미터:")
for key, value in sorted(lgbm_random.best_params_.items()):
    if isinstance(value, float):
        print(f"   {key}: {value:.4f}")
    else:
        print(f"   {key}: {value}")


In [ ]:

# ============================================
# 8. 전체 결과 비교
# ============================================
results_df = pd.DataFrame(lgbm_random.cv_results_)

scoring_cols = [f'mean_test_{score}' for score in scoring.keys()]

comparison_cols = [f'rank_test_{main_score}'] + scoring_cols + ['params']

print("\n" + "="*60)
print(f"📊 상위 10개 파라미터 조합 ({main_score.upper()} 기준)")
print("="*60)
top_10 = results_df[comparison_cols].sort_values(f'rank_test_{main_score}').head(10)

for idx, row in top_10.iterrows():
    rank = int(row[f'rank_test_{main_score}'])
    pr_auc = row[f'mean_test_pr_auc']
    recall = row['mean_test_recall']
    f2 = row['mean_test_f2']
    precision = row['mean_test_precision']
    f1 = row['mean_test_f1']
    
    print(f"\n#{rank} - {main_score.upper()}: {row[f'mean_test_{main_score}']:.4f} | Recall: {recall:.4f}")
    print(f"   F2: {f2:.3f} | Precision: {precision:.3f} | F1: {f1:.3f}")
    
    params = row['params']
    print(f"   Model: lr={params.get('model__learning_rate', 0):.4f}, "
          f"depth={params.get('model__max_depth', 0)}, "
          f"leaves={params.get('model__num_leaves', 0)}")

# ============================================
# 9. 테스트 세트 평가
# ============================================
print("\n" + "="*60)
print("🧪 테스트 세트 평가")
print("="*60)

best_model = lgbm_random.best_estimator_
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)

# ✅ 클래스 1(가격미달)의 확률 추출
y_pred_proba_minority = y_pred_proba[:, 1]  # ✅ 인덱스 1 = 클래스 1 = 가격미달

print(classification_report(
    y_test, y_pred, 
    target_names=['0 (일반가, 다수)', '1 (가격미달, 소수)']
))

print("\n혼동 행렬:")
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm,
                     index=['실제_0 (일반가)', '실제_1 (가격미달)'],
                     columns=['예측_0 (일반가)', '예측_1 (가격미달)'])
print(cm_df)

# ✅ 혼동 행렬 해석 수정
# confusion_matrix 구조:
#              예측_0  예측_1
# 실제_0 (일반가)  TN      FP
# 실제_1 (가격미달)   FN      TP

tn, fp, fn, tp = cm.ravel()

print("\n💼 비즈니스 해석 (소수 클래스=1=가격미달 기준):")
print(f"   ✅ 정확한 분류: {tn + tp}개 ({(tn+tp)/len(y_test)*100:.1f}%)")
print(f"   ⚠️  오탐 (FP): {fp}개 → 일반가을 가격미달로 오판 (재검사 필요)")
print(f"   🚨 미탐 (FN): {fn}개 → 가격미달를 일반가으로 오판 (치명적!)")

# ✅ Recall 계산: TP / (TP + FN)
if tp + fn > 0:
    recall_pct = tp/(tp+fn)*100
    print(f"\n   📊 소수 클래스(1=가격미달) Recall: {recall_pct:.1f}%")
    print(f"      → 가격미달 {tp+fn}개 중 {tp}개 검출 ({fn}개 놓침)")
else:
    print(f"\n   📊 소수 클래스(1=가격미달) Recall: N/A")

# ✅ Precision 계산: TP / (TP + FP)
if tp + fp > 0:
    precision_pct = tp/(tp+fp)*100
    print(f"   📊 소수 클래스(1=가격미달) Precision: {precision_pct:.1f}%")
    print(f"      → 가격미달 판정 {tp+fp}개 중 {tp}개 정답")
else:
    print(f"   📊 소수 클래스(1=가격미달) Precision: N/A")

test_roc_auc = roc_auc_score(y_test, y_pred_proba_minority)
print(f"   📊 ROC-AUC: {test_roc_auc:.4f}")

# ============================================
# 10. 🔥 임계값 조정으로 추가 개선
# ============================================
from sklearn.metrics import precision_recall_curve, average_precision_score

# ✅ pos_label=1 (가격미달) 기준으로 PR 곡선 계산
precisions, recalls, thresholds = precision_recall_curve(
    y_test, y_pred_proba_minority
)

# Recall 50% 보장
target_recall = 0.5
idx = np.where(recalls >= target_recall)[0]

if len(idx) > 0:
    best_idx = idx[np.argmax(precisions[idx])]
    optimal_threshold = thresholds[best_idx]
    
    print("\n" + "="*60)
    print(f"🎯 Recall {target_recall*100}% 보장 시 최적 임계값")
    print("="*60)
    print(f"최적 임계값: {optimal_threshold:.3f} (기본값 0.5)")
    print(f"Precision: {precisions[best_idx]:.3f}")
    print(f"Recall: {recalls[best_idx]:.3f}")
    
    # ✅ 임계값 조정: 확률 >= threshold이면 클래스 1(가격미달)로 예측
    y_pred_adjusted = (y_pred_proba_minority >= optimal_threshold).astype(int)
    
    cm_adj = confusion_matrix(y_test, y_pred_adjusted)
    tn2, fp2, fn2, tp2 = cm_adj.ravel()
    
    print(f"\n조정 효과:")
    print(f"   미탐(FN) 감소: {fn} → {fn2} (개선: {fn-fn2}개)")
    print(f"   오탐(FP) 증가: {fp} → {fp2} (증가: {fp2-fp}개)")
    
    if tp + fn > 0 and tp2 + fn2 > 0:
        print(f"   Recall 개선: {tp/(tp+fn)*100:.1f}% → {tp2/(tp2+fn2)*100:.1f}%")
    
    if fn > fn2:
        print(f"   💡 가격미달를 일반가으로 오판하는 치명적 오류 {fn-fn2}개 감소!")
else:
    print(f"\n⚠️  Recall {target_recall*100}% 달성 불가 (최대 Recall: {recalls.max():.2%})")

# ============================================
# 11. 변수 중요도 확인
# ============================================
print("\n" + "="*60)
print("📊 변수 중요도 Top 10")
print("="*60)

final_model = best_model.named_steps['model']
feature_names = list(X_train.columns)

n_features_used = len(final_model.feature_importances_)
feature_names = feature_names[:n_features_used]

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print(importance_df.head(10).to_string(index=False))

# ============================================
# 12. 교차검증 vs 테스트 성능 비교
# ============================================
print("\n" + "="*60)
print("📈 성능 비교: 교차검증 vs 테스트")
print("="*60)

# ✅ pos_label=1 기본값 사용
test_metrics = {
    'PR-AUC': average_precision_score(y_test, y_pred_proba_minority),
    'Recall': recall_score(y_test, y_pred),
    'F2': fbeta_score(y_test, y_pred, beta=2),
    'Precision': precision_score(y_test, y_pred, zero_division=0),
    'F1': f1_score(y_test, y_pred, zero_division=0),
    'ROC-AUC': test_roc_auc
}

print(f"{'지표':<15} {'교차검증 (CV)':<20} {'테스트':<15} {'차이':<10}")
print("-" * 60)

for metric in ['PR-AUC', 'Recall', 'F2', 'Precision', 'F1', 'ROC-AUC']:
    cv_key = metric.lower().replace('-', '_')
    cv_score = results_df.loc[results_df[f'rank_test_{main_score}'] == 1, f'mean_test_{cv_key}'].values[0]
    test_score = test_metrics[metric]
    diff = test_score - cv_score
    
    diff_str = f"{diff:+.4f}"
    if abs(diff) > 0.05:
        diff_str += " ⚠️"
    
    print(f"{metric:<15} {cv_score:<20.4f} {test_score:<15.4f} {diff_str}")

# ============================================
# 13. 최종 요약
# ============================================
print("\n" + "="*60)
print(f"📝 최종 요약 ({main_score.upper()} 최적화)")
print("="*60)
print(f"✅ 최적 모델 {main_score.upper()} (CV): {lgbm_random.best_score_:.4f}")
print(f"✅ 테스트 Recall: {test_metrics['Recall']*100:.1f}% (가격미달 검출률)")
print(f"✅ 테스트 Precision: {test_metrics['Precision']*100:.1f}% (가격미달 판정 정확도)")
print(f"✅ 테스트 F1: {test_metrics['F1']:.4f}")
print(f"✅ 치명적 오류(미탐 FN): {fn}개 / {tp+fn}개")
print(f"⏱️  총 소요 시간: {actual_time/60:.1f}분")

print("\n💡 핵심 포인트:")
print(f"   - 가격미달(1) {tp+fn}개 중 {tp}개 검출 ({tp/(tp+fn)*100:.1f}%)")
print(f"   - 놓친 가격미달: {fn}개")
print(f"   - 오탐 (일반가→가격미달): {fp}개 (재검사 필요)")

# ============================================
# 14. 모델 + 파라미터 저장
# ============================================
print("\n" + "="*60)
print("💾 모델 및 파라미터 저장")
print("="*60)

save_model_input = input("모델을 저장하시겠습니까? (y/n): ")
if save_model_input.lower() == 'y':
    
    # MODEL_DIR이 정의되지 않은 경우 현재 디렉토리 사용
    try:
        MODEL_DIR = Path(MODEL_DIR)
    except:
        MODEL_DIR = Path('.')
        print(f"⚠️  MODEL_DIR 미정의, 현재 디렉토리 사용: {MODEL_DIR.absolute()}")
    
    MODEL_DIR.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    model_filename = MODEL_DIR / f'best_lgbm_{main_score}_optimized_{timestamp}.pkl'
    params_filename = MODEL_DIR / f'best_params_{main_score}_{timestamp}.json'
    metrics_filename = MODEL_DIR / f'test_metrics_{main_score}_{timestamp}.json'
    results_filename = MODEL_DIR / f'cv_results_{main_score}_{timestamp}.csv'
    
    # 1. 모델 저장
    joblib.dump(best_model, model_filename)
    print(f"✅ 모델 저장: {model_filename}")
    
    # 2. 파라미터 저장
    best_params_serializable = {}
    for key, value in lgbm_random.best_params_.items():
        if isinstance(value, (np.integer, np.floating)):
            best_params_serializable[key] = float(value) if isinstance(value, np.floating) else int(value)
        else:
            best_params_serializable[key] = value
    
    params_info = {
        'best_params': best_params_serializable,
        f'best_{main_score}_score': float(lgbm_random.best_score_),
        'optimization_target': main_score,
        'cv_folds': n_folds,
        'n_iter': n_iter,
        'timestamp': timestamp,
        'class_labels': {
            '0': '일반가 (다수)',
            '1': '가격미달 (소수)'
        },
        'minority_count': int(minority_count),
        'majority_count': int(majority_count),
        'scale_pos_weight': float(scale_pos_weight),
        'features': features,
        'train_test_split': split_date
    }
    
    with open(params_filename, 'w', encoding='utf-8') as f:
        json.dump(params_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 파라미터 저장: {params_filename}")
    
    # 3. 테스트 성능 저장
    test_metrics_serializable = {
        key: float(value) if isinstance(value, (np.floating, np.integer)) else value
        for key, value in test_metrics.items()
    }
    
    metrics_info = {
        'test_metrics': test_metrics_serializable,
        'confusion_matrix': {
            'tn': int(tn),  # 일반가 → 일반가 (정답)
            'fp': int(fp),  # 일반가 → 가격미달 (오탐)
            'fn': int(fn),  # 가격미달 → 일반가 (미탐, 치명적!)
            'tp': int(tp)   # 가격미달 → 가격미달 (정답)
        },
        'business_interpretation': {
            'total_minority': int(tp + fn),      # 실제 가격미달 개수
            'detected_minority': int(tp),        # 검출된 가격미달
            'missed_minority': int(fn),          # 놓친 가격미달 (치명적)
            'recall_percentage': float(tp/(tp+fn)*100) if (tp+fn) > 0 else 0,
            'false_alarms': int(fp)              # 일반가을 가격미달로 오판
        },
        'timestamp': timestamp
    }
    
    with open(metrics_filename, 'w', encoding='utf-8') as f:
        json.dump(metrics_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 테스트 성능 저장: {metrics_filename}")
    
    # 4. CV 결과 저장
    results_df.to_csv(results_filename, index=False, encoding='utf-8')
    print(f"✅ CV 결과 저장: {results_filename}")
    
    print("\n📦 저장된 파일 요약:")
    print(f"1. 모델: {model_filename.name}")
    print(f"2. 파라미터: {params_filename.name}")
    print(f"3. 성능: {metrics_filename.name}")
    print(f"4. CV결과: {results_filename.name}")
    
else:
    print("모델 저장 생략")

print("\n" + "="*60)
print("🎉 모든 작업 완료!")
print("="*60)